In [3]:
import numpy as np
import scipy.special
from itertools import product

T = 1; H = 0.07; steps = 100; npaths = 100000; xi = 0.235**2; eta = 1.9

t = np.linspace(T/steps,T,steps)

v, u = np.zeros(pow(steps,2)), np.zeros(pow(steps,2))

k = 0
for i in product(t, t):
    v[k], u[k] = i
    k = k + 1

index_matrix = np.reshape(np.minimum(v,u),(steps,steps))

def G(H, x):
    F = scipy.special.hyp2f1(1.0,0.5-H,1.5+H,1.0/x)
    return (2*H*(x**(H-0.5))*scipy.special.gamma(0.5+H)*F)/scipy.special.gamma(H+1.5)

WW = np.zeros((steps,steps))
WZ = np.zeros((steps,steps))

ZZ = index_matrix

D = np.sqrt(2.0*H)/(H+0.5)

i = 0
for v in t:
    j = 0
    for u in t:
        if i == j:
            WW[i,j] = u**(2.0*H)
        if i > j:
            x = v/u
            WW[i,j] = u**(2.0*H)*G(H,x)
        if i < j:
            WW[i,j] = 0
        WZ[i,j] = D * (v**(H+0.5)-(v-np.minimum(u,v))**(H+0.5))
        j += 1
    i += 1

WW = WW + WW.T - np.diag(WW.diagonal())

cov = np.block([[WW, WZ],[WZ.T, ZZ]])

mean = np.zeros(steps*2)

#A = scipy.linalg.cholesky(cov, lower=True)
#normal = np.random.randn(steps*2,npaths)
#paths = np.matmul(A, normal)

paths = np.random.multivariate_normal(mean=mean,cov=cov, size=npaths)

In [4]:
paths.shape

(100000, 200)

In [5]:
paths[:,:steps].T.shape

(100, 100000)